In [2]:
%matplotlib inline
import ee 
import geemap

import matplotlib.pyplot as plt

## will need to import the gee functions 
import os 
os.chdir(r'D:\RT_temperaturePrivate\Analysis\RTT_Pycharm\TemperatureFuncs')
from LandsatGEE import extractLS8,extractLS9
import GEE_GenericFuncs
from AsterGEE import extractAster

pp = ee.FeatureCollection("WRI/GPPD/power_plants") 


In [3]:
ee.Initialize()

## Process
* Will try and get as close to one image a month at this site
* Will double check these then upload them one at a time
* Will then download the temperature tifs 

# DataSets

## Confirmed dates - PS
* LS8: 8th March == i8_1
* LS8 18th April == i8_2
* LS9 12th May == i9_3
* LS9 20th June == i9_4
* Aster: 8th of July == iA_5
* LS9 16th August == i9_6
* LS8 18th October == i8_7



to check them we load them individually onto a map with a bounding box over the area we want


In [4]:
# set up
task = 'extractValues_atmosphericallyCorrected'
Ngeometry = ee.FeatureCollection('projects/ee-samuelvalman/assets/P3RT/Rhone')
geometry = Ngeometry.geometry()

In [5]:

#### I8_1 #####
date_start = '2022-03-07'
date_end = '2022-03-09'
I8_1 =  extractLS8(geometry, task=task, date_start=date_start, date_end=date_end)
#### I8_2 #####
date_start = '2022-04-17'
date_end = '2022-04-19'
I8_2 =  extractLS8(geometry, task=task, date_start=date_start, date_end=date_end)
#### I9_3 #####
date_start = '2022-05-11'
date_end = '2022-05-13'
I9_3 =  extractLS9(geometry, task=task, date_start=date_start, date_end=date_end)
#### I9_4 #####
date_start = '2022-06-19'
date_end = '2022-06-21'
I9_4 =  extractLS9(geometry, task=task, date_start=date_start, date_end=date_end)
#### Ia_5 #####
date_start = '2022-07-08'
date_end = '2022-07-09'
Ia_5 =  extractAster(geometry, task=task, date_start=date_start, date_end=date_end)
#### I9_6 #####
date_start = '2022-08-15'
date_end = '2022-08-17'
I9_6 =  extractLS9(geometry, task=task, date_start=date_start, date_end=date_end)
#### I8_7 #####
date_start = '2022-10-17'
date_end = '2022-10-19'
I8_7 =  extractLS8(geometry, task=task, date_start=date_start, date_end=date_end)

# Plot these 

In [6]:
visParams = visParams = {'bands': ['wst'],  'min': 20,  'max': 34, 'palette': ['blue', 'blue', 'blue', 'cyan', 'green', 'yellow', 'red', 'red','red']}

In [7]:
outline = [[[4.734421, 45.382416],
   [4.734421, 45.408333],
   [4.764977, 45.408333],
   [4.764977, 45.382416],
   [4.734421, 45.382416]]]
outline = ee.Geometry.Polygon(outline) #Saint Alban

In [8]:
imlist = [I8_1, I8_2, I9_3, I9_4, Ia_5, I9_6, I8_7]
ic = imlist[6]
print(ic.size().getInfo())
im = ic.select('wst') #.first()

Map = geemap.Map()
Map.addLayer(im, vis_params=visParams, name='s')
# Map.addLayer(ic.first(), vis_params={'bands':['SR_B2', 'SR_B3', 'SR_B4']}, shown=False)
# Map.addLayer(pp)
# Map.addLayer(outline)
Map.centerObject(outline)
# Map

1


In [9]:
# WaterMasks
wm = ee.ImageCollection('projects/ee-samuelvalman/assets/P3RT/waterMasks')

# download 

In [12]:
imlist = [I8_1, I8_2, I9_3, I9_4, Ia_5, I9_6, I8_7]
nm = ['2022_03_08', '2022_04_18', '2022_05_12', '2022_06_20','2022_07_08','2022_08_16', '2022_10_18']
for i in range(len(imlist)):
    ic = imlist[i]
    im = ic.select('wst').median()
    im2 = im.reproject(crs='EPSG:4326', scale=30)

    geemap.ee_export_image_to_drive(im2, description=nm[i],  region=outline, scale=30, crs=None, crsTransform=None, maxPixels=None)